# 线性可分支持向量机算法

![支持向量机](../images/支持向量机/最大间隔法.jpg)

![存在唯一性](../images/支持向量机/存在唯一性.jpg)
![存在唯一性](../images/支持向量机/存在唯一性2.jpg)
![存在唯一性](../images/支持向量机/存在唯一性3.jpg)

![对偶算法](../images/支持向量机/对偶算法.jpg)
![对偶算法](../images/支持向量机/对偶算法2.jpg)
![对偶算法](../images/支持向量机/对偶算法3.jpg)
![对偶算法](../images/支持向量机/对偶算法4.jpg)

# 线性支持向量机算法

![线性支持向量机](../images/支持向量机/线性支持向量机.jpg)
![线性支持向量机](../images/支持向量机/线性支持向量机2.jpg)
![线性支持向量机](../images/支持向量机/线性支持向量机3.jpg)
![线性支持向量机](../images/支持向量机/线性支持向量机4.jpg)

![合叶损失函数](../images/支持向量机/合叶损失函数.jpg)
![合叶损失函数](../images/支持向量机/合叶损失函数2.jpg)

![核技巧](../images/支持向量机/核技巧.jpg)
![核技巧](../images/支持向量机/核技巧2.jpg)
![核技巧](../images/支持向量机/核技巧3.jpg)
![核技巧](../images/支持向量机/核技巧4.jpg)
![核技巧](../images/支持向量机/核技巧5.jpg)

# 序列最小优化算法（SMO算法）

![SMO算法](../images/支持向量机/SMO算法.jpg)
![SMO算法](../images/支持向量机/SMO算法2.jpg)
![SMO算法](../images/支持向量机/SMO算法3.jpg)
![SMO算法](../images/支持向量机/SMO算法4.jpg)
![SMO算法](../images/支持向量机/SMO算法5.jpg)
![SMO算法](../images/支持向量机/SMO算法6.jpg)
![SMO算法](../images/支持向量机/SMO算法7.jpg)

## SMO算法的求解

![SMO算法](../images/支持向量机/SMO算法8.jpg)
![SMO算法](../images/支持向量机/SMO算法9.jpg)
![SMO算法](../images/支持向量机/SMO算法10.jpg)
![SMO算法](../images/支持向量机/SMO算法11.jpg)
![SMO算法](../images/支持向量机/SMO算法12.jpg)
![SMO算法](../images/支持向量机/SMO算法13.jpg)
![SMO算法](../images/支持向量机/SMO算法14.jpg)
![SMO算法](../images/支持向量机/SMO算法15.jpg)
![SMO算法](../images/支持向量机/SMO算法16.jpg)

In [1]:
import sys
import time
import random
import numpy as np
from collections import Counter
from heapq import heappush, heappop
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
data = load_breast_cancer()
X, y = data['data'], data['target']
train_X, test_X, train_y, test_y = train_test_split(X, y)
print('train size: %s' % len(train_X))
print('test size: %s' % len(test_X))

train size: 426
test size: 143


In [3]:
class SVM:
    def __init__(self, train_X, train_y, sigma=10, C=200, tol=0.001, iters=10):
        self.train_X = np.mat(train_X)
        self.train_y = train_y
        self.m, self.n = np.shape(self.train_X)
        self.sigma = sigma
        self.C = C
        self.tol = tol
        self.iters = iters
        self.kernal = self.__calcKernel()
        self.bias = 0
        self.alpha = [0] * self.m
        self.E = [0 for i in range(self.m)] 
        self.__train()
        self.supportVecIndex = [i for i in range(self.m) if (self.alpha[i] > 0)]
        
        
    def __calcKernel(self):
        k = [[0 for i in range(self.m)] for j in range(self.m)]
        for i in range(self.m):
            X = self.train_X[i, :]
            for j in range(i, self.m):
                Z = self.train_X[j, :]
                result = (X - Z) * (X - Z).T
                result = np.exp(-1 * result / (2 * self.sigma**2))
                k[i][j] = result
                k[j][i] = result
                
        return k
    
    def __calc_gxi(self, i):
        gxi = 0
        for j in range(self.m):
            gxi += self.alpha[i] * self.kernal[j][i] * self.train_y[i]
        return gxi + self.bias
    
    def __isSatisfyKKT(self, i):
        gxi = self.__calc_gxi(i)
        yi = self.train_y[i]
        if (abs(self.alpha[i]) < self.tol) and (yi * gxi >= 1):
            return True
        elif (abs(self.alpha[i] - self.C) < self.tol) and (yi * gxi <= 1):
            return True
        elif (self.alpha[i] > -self.tol) and (self.alpha[i] < (self.C + self.tol)) and (
                                            abs(yi * gxi - 1) < self.tol):
            return True
        return False
    
    def __calcEi(self, i):
        gxi = self.__calc_gxi(i)
        return gxi - self.train_y[i]
    
    def __get_alpha2(self, E1, i):
        E2 = 0
        max_E1_sub_E2 = -1
        maxIndex = -1
        nonZero = [i for i, Ei in enumerate(self.E) if Ei != 0]
        for j in nonZero:
            E2_tmp = self.__calcEi(j)
            if abs(E1 - E2_tmp) > max_E1_sub_E2:
                max_E1_sub_E2 = abs(E1 - E2_tmp)
                E2 = E2_tmp
                maxIndex = j

        if maxIndex == -1:
            maxIndex = i
            while maxIndex == i:
                maxIndex = int(random.uniform(0, self.m))
            E2 = self.__calcEi(maxIndex)
        return E2, maxIndex  
    
    def __train(self):
        step = 0
        convergence = False
        while step < self.iters or not convergence:
            changed_count = 0
            for i in range(self.m):
                if not self.__isSatisfyKKT(i):
                    E1 = self.__calcEi(i)
                    E2, j = self.__get_alpha2(E1, i)
                    y1 = self.train_y[i]
                    y2 = self.train_y[j]
                    alpha_old1 = self.alpha[i]
                    alpha_old2 = self.alpha[j]
                    if y1 != y2:
                        L = max(0, alpha_old2 - alpha_old1)
                        H = min(self.C, self.C + alpha_old2 - alpha_old1)
                    else:
                        L = max(0, alpha_old2 + alpha_old1 - self.C)
                        H = min(self.C, alpha_old2 + alpha_old1)
                    
                    if L == H:
                        continue
                    k11 = self.kernal[i][i]
                    k12 = self.kernal[i][j]
                    k21 = self.kernal[j][i]
                    k22 = self.kernal[j][j]
                    
                    alpha_new2_unc = alpha_old2 + y2 * (E1 - E2) / (k11 + k22 - 2 * k12)
                    if alpha_new2_unc > H:
                        alpha_new2 = H
                    if alpha_new2_unc < L:
                        alpha_new2 = L
                    if L <= alpha_new2_unc <= H:
                        alpha_new2 = alpha_new2_unc
                    
                    alpha_new1 = alpha_old1 + y1 * y2 * (alpha_old2 - alpha_old1)
                    
                    b1_new = -E1 - y1*k11*(alpha_new1 - alpha_old1) - y2*k21*(alpha_new2 - alpha_old2) + self.bias
                    b2_new = -E2 - y1*k12*(alpha_new1 - alpha_new2) - y2*k22*(alpha_new2 - alpha_old2) + self.bias
                    
                    if (0 < alpha_new1 < self.C) and (0 < alpha_new2 < self.C):
                        self.bias = b1_new
                        
                    else:
                        self.bias = (b1_new + b2_new) / 2
                        
                    self.alpha[i] = alpha_new1
                    self.alpha[j] = alpha_new2
                    
                    self.E[i] = self.__calcEi(i)
                    self.E[j] = self.__calcEi(j)
                    if abs(alpha_new2 - alpha_old2) < 0.00001:
                        changed_count += 1
                        
                convergence = True if not changed_count else False
            step += 1
            print(step)
    
    def __calcSinglKernel(self, x1, x2):
        result = (x1 - x2) * (x1 - x2).T
        result = np.exp(-1 * result / (2 * self.sigma ** 2))
        return np.exp(result)
    
    
    def __predict(self, x):
        result = 0
        for i in self.supportVecIndex:
            tmp = self.__calcSinglKernel(self.train_X[i, :], np.mat(x))
            result += self.alpha[i] * self.train_y[i] * tmp
        result += self.b
        return 0 if result < 0 else 1
    
    def predict(self, test_X):
        preds = []
        for i in test_X:
            preds.append(self.__predict(i))
            
        return preds

In [4]:
svm = SVM(train_X, train_y)

1


KeyboardInterrupt: 